# JMeter results (.csv) file analysis notebook
- copy the results .csv file to the docker mounted volume which is the directory from where you started this container

- make sure we have column header name as the 1st rown in the below order _timeStamp,elapsed,label,responseCode,responseMessage,threadName,dataType,success,failureMessage,bytes,sentBytes,grpThreads,allThreads,Latency,Connect_

- transaction controller name have the workspace/tenant id suffixed as _<workspaceid/tenantid>

> *Dataframe is sensitive to whitespaces, make sure in the label and other places there are not leading/trailing whitespaces at all*


In [ ]:
import pandas as pd
import seaborn.objects as so
import seaborn as sns

file_path = '/home/ubuntu/work/jmeter_disconnect_results_jmeter-result.csv'
df = pd.read_csv(file_path)

df['timeStamp'] = pd.to_datetime(df['timeStamp'], unit='ms')

# filter out only the transaction level data points
df_filtered = df[df.responseMessage.str.match('Number of samples in transaction.*',na=False)]

# label name have tenant/workspace as suffix, split the label column into 2 with a regex and 
# the new column will be added as workspace dimension
df_filtered[['label','workspace']] = df_filtered['label'].str.extract('(.*)_(.+)$',expand=True)

# generate a new column lat_perc_of_resp_time expressing latency as % of reponse time
df_filtered ['lat_perc_of_resp_time'] = 100*df_filtered['Latency']/df_filtered['elapsed']

# generate a new column conn_perc_of_resp_time expressing connect time as % of reponse time
df_filtered ['conn_perc_of_resp_time'] = 100*df_filtered['Connect']/df_filtered['elapsed']


## Total test duration

In [ ]:
df.reset_index().set_index('timeStamp',inplace=True)
print (df.timeStamp.iloc[-1] -df.timeStamp.iloc[0])

## Elapsed/response time summary
breakup by workspace/tenantid

In [ ]:
df_filtered.groupby('workspace')['elapsed'].describe ()

## Downsampling elapsed, Latency, Connect column
5 sec interval with aggregation function max applied

In [ ]:
# convert the index to DatetimeIndex from RangeIndex for aiding with downsampling etc
df_downsample = df.copy()
df_downsample.set_index('timeStamp',inplace=True)
df_downsample = df_downsample[["elapsed","Latency","Connect"]].resample("5s").max()
df_downsample.plot()

## Individual _elapsed/response time_ plot per transaction (facet col=label) and grouped by workspace (color=workspace)

In [ ]:

(
    so.Plot(df_filtered, x="timeStamp", y="elapsed", color="workspace")
    .facet(col = 'label',wrap=9)
    .add(so.Line())
    .layout(size=(90, 15),engine="constrained")
)


## Individual _latency time_ plot per transaction (facet col=label) and grouped by workspace (color=workspace)

In [ ]:
(
    so.Plot(df_filtered, x="timeStamp", y="Latency", color="workspace")
    .facet(col = 'label',wrap=9)
    .add(so.Line())
    .layout(size=(90, 15),engine="constrained")
)

## Individual _Connect time_ plot per transaction (facet col=label) and grouped by workspace (color=workspace)

In [ ]:
(
    so.Plot(df_filtered, x="timeStamp", y="Connect", color="workspace")
    .facet(col = 'label',wrap=9)
    .add(so.Line())
    .layout(size=(90, 15),engine="constrained")
)

## Filter elapsed/response time > 5sec, plot transaction across workspace
Aggregation max/min/mean/count

In [ ]:
p_table = pd.pivot_table( data=df_filtered, 
                        index=['label'], 
                        columns=['workspace'], 
                        values='elapsed',
                        aggfunc=['max','min', 'mean', 'count'],
                        fill_value="None")

#p_table
p_table[p_table["mean"] > 5000]

## Max elapsed/response time and threadcount summary across workspaces
Aggregation max for both elapsed/response time and threadcount/allThreads
Filter only transactions with response time > 5sec
Ref: https://datagy.io/python-pivot-tables/

In [ ]:
p_table = pd.pivot_table( data=df_filtered, 
                        index=['label'], 
                        values=['elapsed','allThreads'],
                        aggfunc={'elapsed':'max','allThreads':'max'},
                        fill_value="None")

#p_table
print(p_table[p_table["elapsed"] > 5000])

## Regression/trend analysis for elapsed/response time
Original data resampled to 5 sec interval 

In [ ]:
import numpy as np

df_fit = df_filtered.copy()

# Convert 'timeStamp' column to datetime
df_fit['timeStamp'] = pd.to_datetime(df_fit['timeStamp'])

# Convert datetime to numeric representation (Unix epoch time in this case)
df_fit['date_ordinal'] = df_fit['timeStamp'].astype(int) // 10**9

# Fit a linear regression line using 'x_numeric' and 'y' columns and get the slope
coefficients = np.polyfit(df_fit['date_ordinal'], df_fit['elapsed'], 1)
slope = coefficients[0]

# Optional: You can convert the slope back to a human-readable format
# Assuming the slope is in units per second (m/s in this case)
slope_human_readable = slope * (10**9)

print("Slope:", slope)
print("Slope (human-readable):", slope_human_readable)


In [ ]:
## Regression/trend analysis for elapsed/response time (using sklearn package)
Original data resampled to 5 sec interval 

In [ ]:
from sklearn import linear_model
import datetime as dt

df_reg = df_filtered.copy()

df_reg = pd.DataFrame(df_reg, columns=['timeStamp', 'elapsed'])

df_reg['date_ordinal'] = df_fit['timeStamp'].astype(int) // 10**9

reg = linear_model.LinearRegression()
reg.fit(df_reg['date_ordinal'].values.reshape(-1, 1), df_reg['elapsed'].values)
reg.coef_


In [ ]:
## Plot elapsed/response time trend graph
_It takes some time to load all the individual plots_

In [ ]:
# Using Hue to Add an Additional Variable to Seaborn lmplot()
import matplotlib.pyplot as plt
df_mat_plot = df_filtered.copy()

df_mat_plot = pd.DataFrame(df_mat_plot, columns=['timeStamp', 'elapsed','label', 'workspace'])

df_mat_plot['date_ordinal'] = df_mat_plot['timeStamp'].astype(int) // 10**9

#sns.lmplot(data=df_mat_plot, x='date_ordinal', y='elapsed', hue='workspace')

#sns.regplot(data=df_mat_plot, x='date_ordinal', y='elapsed')

sns.lmplot(data=df_mat_plot, x='date_ordinal', y='elapsed', col='workspace', row='label')

plt.show()